# Transfer Learning

In [1]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications.mobilenet import decode_predictions
import numpy as np
import os

In [2]:
m = MobileNet(input_shape=(224, 224, 3), include_top=False)

In [44]:
def read_images(path):
    """Returns a list of input images from the given path"""
    images = []
    for fn in os.listdir(path):
        if fn.startswith('.'):
            continue
        im = image.load_img(path + '/' + fn, target_size=(224, 224))
        im = image.img_to_array(im)
        X = im.reshape(224, 224, 3)
        images.append(X)
        
    X = np.array(images)
    X = preprocess_input(X)
    return X

#### 1. read all duck + car images

In [4]:
car_images = read_images('car')
car_images.shape

(25, 224, 224, 3)

In [5]:
duck_images = read_images('duck')
duck_images.shape

(26, 224, 224, 3)

#### 2. calculate feature vector for all images
use `m.predict(X)`

In [6]:
duck_images[0].shape

(224, 224, 3)

In [7]:
m.predict(duck_images[0].reshape(1,224,224,3)).shape

(1, 7, 7, 1024)

In [8]:
ducks = m.predict(duck_images)
ducks.shape

(26, 7, 7, 1024)

In [9]:
cars = m.predict(car_images)
cars.shape

(25, 7, 7, 1024)

In [10]:
X = np.vstack((cars, ducks))
X.shape

(51, 7, 7, 1024)

In [11]:
m.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

#### 3. create a vector with labels

In [12]:
ones = np.ones(26)
zeros = np.zeros(25)
y = np.hstack((ones,zeros))
y.shape

(51,)

#### 4. inspect shapes

In [13]:
X.shape

(51, 7, 7, 1024)

In [14]:
y.shape

(51,)

#### 5. train-test-split

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
xtrain, xtest, ytrain, ytest = train_test_split(X,y)

In [17]:
xtrain.shape

(38, 7, 7, 1024)

In [18]:
ytrain.shape

(38,)

In [19]:
xtrain.reshape(xtrain.shape[0],7*7*1024).shape

(38, 50176)

#### 6. train a LogReg model
determine a training, test and cross-validation score

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
log = LogisticRegression()

In [22]:
log.fit(xtrain.reshape(xtrain.shape[0],7*7*1024), ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
log.score(xtrain.reshape(xtrain.shape[0],7*7*1024),ytrain)

1.0

In [24]:
log.score(xtest.reshape(xtest.shape[0],7*7*1024),ytest)

1.0

In [25]:
log.predict(xtest.reshape(xtest.shape[0],7*7*1024))

array([1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0.])

In [26]:
ytest

array([1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0.])

### 7. train a logreg model without MobileNet

In [27]:
car_images.shape

(25, 224, 224, 3)

In [28]:
X2 = np.vstack((car_images, duck_images))
X2.shape

(51, 224, 224, 3)

In [29]:
X2.reshape(51,224*224*3).shape

(51, 150528)

In [30]:
x2train, x2test, y2train, y2test = train_test_split(X2.reshape(51,224*224*3), y)

In [31]:
x2train.shape

(38, 150528)

In [32]:
log2 = LogisticRegression()

In [33]:
log2.fit(x2train,y2train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
log2.score(x2train,y2train)

1.0

In [35]:
log2.score(x2test,y2test)

1.0

In [36]:
y2test

array([0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0.])

In [37]:
log2.predict(x2test)

array([0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0.])

## Read Max's duck image

In [38]:
new_duck = read_images('extra')
new_duck.shape

(1, 224, 224, 3)

### Score Mobile Net + logistic regression model

In [39]:
output = m.predict(new_duck)
log.predict(output.reshape(1,7*7*1024))
#output.shape

array([1.])

### Score simple logistic regression model

In [40]:
log2.predict(new_duck.reshape(1,224*224*3))

array([1.])

In [41]:
log2.predict(duck_images[0].reshape(1,224*224*3))

array([1.])

## Read more duck images

In [45]:
new_ducks = read_images('jpgs')
new_ducks.shape

(12, 224, 224, 3)

### Score simple logistic regression model

In [46]:
log2.score(new_ducks.reshape(12,224*224*3), np.ones(12))

0.6666666666666666

### Score Mobile Net + logistic regression model

In [47]:
output = m.predict(new_ducks)
#log.predict(output.reshape(1,7*7*1024))
output.shape

(12, 7, 7, 1024)

In [48]:
log.score(output.reshape(12,7*7*1024),np.ones(12))

1.0